## Background

Lots of people experience fiddly behavior when using LLMs.  For example:

<blockquote class="twitter-tweet"><p lang="en" dir="ltr">Unironically I found this to be very helpful when prompting LLMs. Giving them spaces and new lines <a href="https://t.co/vVuxcCuDzB">pic.twitter.com/vVuxcCuDzB</a></p>&mdash; anton (@abacaj) <a href="https://twitter.com/abacaj/status/1728190808191537604?ref_src=twsrc%5Etfw">November 24, 2023</a></blockquote> <script async src="https://platform.twitter.com/widgets.js" charset="utf-8"></script>

If you aren't careful, these can be very hard to debug.  This is because of the subtle ways tokenizers work that is not always easy to tell by looking at the text.  

## Example

The below example demonstrates how things can get confusing and can drift between training and inference time.


In [1]:
#|echo: false
from transformers import AutoTokenizer
from functools import partial
model_id = 'Open-Orca/Mistral-7B-OpenOrca'
tok = AutoTokenizer.from_pretrained(model_id)

/home/hamel/mambaforge/envs/honeycomb/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
enc = partial(tok.encode, add_special_tokens=False)
dec = partial(tok.decode)

#### Many frameworks do prompt construction by concatenating tokens

Popular frameworks like [axolotl](https://github.com/OpenAccess-AI-Collective/axolotl) construct prompts by concatenating tokens instead of strings.[^1]  It is reasonable to decode the training data to check what the prompt template is:

[^1]: This is for good reason, as masking must also be done at the token level.

For example, a prompt may be constructed like this:

In [3]:
axolotl = enc('Ok\n') + enc('<|im_start|>')
print(dec(axolotl))

Ok
<|im_start|>


#### Let's say you have an inference server

It's common for inference servers to assemble the prompt for you.  The below looks Like it should be fine, right? 

In [4]:
def inf_server(inp): 
    return f'{inp}\n<|im_start|>'

srv = inf_server('Ok')
print(srv)

Ok
<|im_start|>


#### Drift between your server and the way the model is trained

Wrong!  Notice the difference in the decoding of the prompt vs the training data.  This is a subtle bug that can be hard to debug.

In [5]:
print(f'axolotl training data:  {axolotl}')
print(f"your server's decoding: {enc(srv)}")

axolotl training data:  [6504, 13, 32001]
your server's decoding: [6504, 32001]


## Solution

Decode your inference data right before your forward pass.  For example, you'll notice the newline is missing if you do this.  This is one way to tell that something fishy is going on.

In [6]:
dec(enc(srv))

'Ok<|im_start|>'

In [7]:
#|include: false
from transformers import AutoModelForCausalLM
model = AutoModelForCausalLM.from_pretrained(model_id)

/home/hamel/mambaforge/envs/honeycomb/lib/python3.11/site-packages/torch/cuda/__init__.py:138: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1699449201450/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
Loading checkpoint shards: 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  3.00s/it]


In [35]:
#|include: false
import torch
import pandas as pd


def prompt_w_activations(input_ids):
    "See activations for all tokens in the pre-fill"
    ids = torch.tensor([input_ids])
    with torch.no_grad():
        outputs = model(ids)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1).squeeze()

    data = []
    for i,idx in enumerate(ids.squeeze()):
        argmax = torch.argmax(probs[i, :]).item()
        data.append({'id': idx.item(), 
                     'argmax_id': argmax,
                     'argmax_prob': probs[i, argmax].item(),
                     'argmax_tok': dec([argmax]),
                     'prob': probs[i,idx].item(),
                     'token': dec([idx.item()])})
    return pd.DataFrame(data)